# 7章 マルチクラス分類

In [39]:
from sklearn.metrics import accuracy_score

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel

本章では、文章のマルチクラス分類を行う。マルチラベル分類は、複数のカテゴリに所属する文章を分類するタスクである。<br>
ラベルには、Multi-Hotベクトルと呼ばれるベクトルをあてる。これは、$(0, 1, 1, 0)$といった、One-Hotベクトルの発展的なかたちで、各要素(ラベル)に対しそれぞれフラグがたったものである。$(0, 1, 1, 0)$は、ある文章が4つのカテゴリのうちカテゴリ2と3にあてはまることを意味する。

まずは`transformers.BertModel`クラスをベースに`BertForSequenceClassificationMultiLabel`クラスを実装し、その挙動を確認していく。実装に関して、シングルラベル分類とはいくつか異なる点が存在するため注意する。

1. 3つ以上のカテゴリを持つシングルラベル分類と違い、損失関数には`BinaryCrossEntropyLoss`を用いる。<br>
Multi-Hotベクトルは各カテゴリに対して`0 or 1`で表現されるためである。
2. 各カテゴリ(Multi-Hotベクトルの各要素)に対し文章が当てはまる確率を出力するため、>50%の際に1とする実装が必要である。

本章では、モデルの最終層の出力をすべてのトークンに対し平均化し、線形変換を適用したものをスコアとする。<br>
平均化にあたり、文章長を調整する`[PAD]`トークンを削除する必要がある。`[PAD]`トークンは`encoding`で得られる`attention_mask`で`0`が与えられるため、`attention_mask`が`1`のトークンで平均を取るようにする。

In [40]:
# 7-4

class BertForSequenceClassificationMultiLabel(torch.nn.Module):
    def __init__(self, model_name, num_labels):
        super().__init__()

        # BERTモデルの読み込み
        self.model = BertModel.from_pretrained(model_name)
        # 線形結合の初期化
        self.linear = torch.nn.Linear(
            self.model.config.hidden_size, num_labels
        )
    
    def forward(
        self, 
        input_ids: torch.Tensor=None, 
        attention_mask: torch.Tensor=None, 
        token_type_ids: torch.Tensor=None,
        labels: torch.Tensor=None,
    ):
        # モデルの最終層の出力
        model_output = self.model(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids,
        )
        # (batch_size, トークン数, 隠れ層数768) のtorch.Tensorを得る
        # ここでは(2, 17, 768)
        last_hidden_state = model_output.last_hidden_state

        # [PAD]トークン以外で平均を取る
        # attention_maskはtokenizerより渡された
        # (バッチサイズ, トークン数) のOne-Hotベクトルのtorch.Tensor
        # ここでは(2, 17)
        # torch.Tensor.unsqueeze(axis)メソッドで新規に1の次元を挿入する
        # (2, 17) -> (2, 17, 1)
        # torch.Tensor.unsqueeze(axis=2)と同義
        # last_hidden_stateが3次元ベクトルなので合わせる
        # torch.Tensor.sum(axis)で次元に沿って2次元目で足し算する
        # (2, 17, 768) -> (2, 768)
        # attention_mask.sum(axis=1)でattention_maskが1となる数を取得する
        # keepdim=Trueにすることで、sum()で次元が(2, 17) -> (2,)になるのを防ぎ
        # 2次元テンソルの形 今回は(2, 1) を維持する
        averaged_hidden_state = (
            last_hidden_state * attention_mask.unsqueeze(axis=-1)
        ).sum(axis=1) / attention_mask.sum(axis=1, keepdim=True)

        # 線形結合する
        scores = self.linear(averaged_hidden_state)

        # lossの計算
        output = {"logits": scores}
        if labels is not None:
            loss = torch.nn.BCEWithLogitsLoss()(scores, labels.float()) # float型に変換する必要がある
            output["loss"] = loss

        # 属性でアクセスできるようにする
        # lossにはoutput.lossでアクセスできるようになる
        output = type("bert_output", (object,), output)
        return output

<br>

モデルと文章、ラベルを定義する。

In [41]:
# 7-5, 7-6

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassificationMultiLabel(
    MODEL_NAME, num_labels=2
)
# model = mode.cuda()

# [CLS] + 本文15トークン + [SEP] の最大17トークン
# ['今日', 'の', '仕事', 'は', 'うまく', 'いっ', 'た', 'が', '、', '体調', 'が', 'あまり', '良く', 'ない', '。']
text_lst = [
    '今日の仕事はうまくいったが、体調があまり良くない。', 
    '昨日は楽しかった。'
]

# 2次元目は[負の感情, 正の感情]
labels_lst = [
    [1, 1],
    [0, 1]
]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<br>

トークナイザを定義し予測を行う。

In [42]:
encoding = tokenizer(text_lst, padding='longest', return_tensors='pt')
encoding["labels"] = torch.tensor(labels_lst)
# encoding = {key: torch.tensor(value) for key, value in encoding.items()}

output = model(**encoding)
scores = output.logits
labels_pred = (scores > 0).int() # スコアが>0ならTrue、そうでないならFalse
accuracy = accuracy_score(labels_lst, labels_pred)

In [43]:
print(f"y_true: {labels_lst}")
print(f"y_pred: {labels_pred}")
print(f"accuracy: {accuracy}")
print(f"loss: {output.loss:.3f}")

y_true: [[1, 1], [0, 1]]
y_pred: tensor([[0, 1],
        [0, 1]], dtype=torch.int32)
accuracy: 0.5
loss: 0.619


1文目はラベル`[1, 1]`に対し`[0, 1]`と予測してハズレ<br>
2文目はラベル`[0, 1]`に対し`[0, 1]`と予測してアタリ<br>
総合でAccuracyは0.5となる。

## 7.5 chABSA-datasetでマルチクラス分類

本章では、TIS株式会社が公開している、[上場企業の有価証券報告書から作成されたマルチラベルデータセット`chABSA-dataset`](https://github.com/chakki-works/chABSA-dataset)を用いる。

このデータセットは、「ネガティブ」「ポジティブ」「ニュートラル」という3クラスを用いる。文章に対してそれぞれのクラスに該当する表現があると、カテゴリーと何を対象としているかをラベル付けしている。

In [44]:
import os
import glob
import json
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl

In [ ]:
os.makedirs("data/chapter7", exist_ok=True)

In [ ]:
!curl --output "data/chapter7/chABSA-dataset.zip" "https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip"
!unzip -q -d "data/chapter7" "data/chapter7/chABSA-dataset.zip"

<br>

それぞれのファイルは`chABSA-dataset/e*****_ann.json`で、全体で230ある。<br>
まずは中身を見てみる。

In [ ]:
# 7-9

data = json.load(open("data/chapter7/chABSA-dataset/e00030_ann.json"))

In [ ]:
data['sentences'][0]

`sentences`内に文章ID`sentence_id`、文章`sentence`、ラベルの集合`opinions`が格納されている。

これをMult-Hotベクトルに変換する。

In [46]:
# 7-10

file_path_lst = glob.glob("data/chapter7/chABSA-dataset/*.json")

# valueがそのままlistのインデックスになるように
category_id = {"negative": 0, "neutral": 1, "positive": 2}

dataset = []
for file_path in file_path_lst:
    data = json.load(open(file_path))
    for sentence in data['sentences']:
        text = sentence['sentence']
        labels = [0, 0, 0]
        for opinion in sentence['opinions']:
            # 各opinionのpolarityをcategory_idのkeyとして
            # category_idのvalueをlabelsのインデックスに見立てる
            labels[category_id[opinion['polarity']]] = 1
        sample = {"text": text, "labels": labels}
        dataset.append(sample)

In [47]:
dataset[8]

{'text': '利益面では、メキシコ子会社の量産準備費用の増加や円高基調等のマイナス要素もありましたが、アジア及び国内子会社の原価改善等の効果が上回りました',
 'labels': [1, 0, 1]}

## 7.6 ファインチューニングと性能評価

まず、トークナイザとデータセットを作成する。<br>
データセットのサイズは`train`、`val`、`test`がそれぞれ$6:2:2$とする。

In [48]:
# 7-12

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

max_length = 128 # 6章と同じく簡略化のため
dataset_for_loader = []
labels_lst = []
for data in dataset:
    text = data['text']
    labels = data['labels']
    encoding = tokenizer(
        text, max_length=max_length, padding='max_length', truncation=True
    )
    encoding["labels"] = labels
    # あとでPytorch Lightningに読み込ませるため.cuda()は必要なし
    encoding = {key: torch.tensor(value) for key, value in encoding.items()}
    dataset_for_loader.append(encoding)
    labels_lst.append(labels)

In [49]:
dataset_tmp, test_dataset, labels_tmp, _ = train_test_split(
    dataset_for_loader, labels_lst, test_size=0.2, shuffle=True, stratify=labels_lst
)
train_dataset, val_dataset = train_test_split(
    dataset_tmp, test_size=0.25, shuffle=True, stratify=labels_tmp
)
train_dataloader = DataLoader(train_dataset, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=256)
test_dataloader = DataLoader(test_dataset, batch_size=256)

<br>

上で作成した`BertForSequenceClassificationMultiLabel`をラップする`BertForSequenseClassificationMultiLabel_pl`モデルを構築する。

In [51]:
# 7-13

class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):
    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters()
        self.model = BertForSequenceClassificationMultiLabel(
            model_name=model_name, num_labels=num_labels
        )

    def training_step(self, batch, batch_idx):
        output = self.model(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        output = self.model(**batch)
        loss = output.loss
        self.log('val_loss', loss)

    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.model(**batch)
        scores = output.logits
        labels_pred = (scores > 0).int()
        if torch.cuda.is_available():
            accuracy = accuracy_score(labels.cpu().numpy(), labels_pred.cpu().numpy())
        else:
            accuracy = accuracy_score(labels.numpy(), labels_pred.numpy())
        self.log('accuracy', accuracy)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [52]:
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath="model/"
)
trainer = pl.Trainer(
    devices=1, accelerator='gpu', max_epochs=5, callbacks=[checkpoint]
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [53]:
model = BertForSequenceClassificationMultiLabel_pl(
    model_name, num_labels=3, lr=1e-5
)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [54]:
trainer.fit(model, train_dataloader, val_dataloader)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /content/model exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                                    | Params
------------------------------------------------------------------
0 | model | BertForSequenceClassificationMultiLabel | 110 M 
------------------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
442.479   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [55]:
test = trainer.test(dataloaders=test_dataloader)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/model/epoch=2-step=345-v1.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/model/epoch=2-step=345-v1.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9109477124183006     │
└───────────────────────────┴───────────────────────────┘

In [56]:
print(test)

[{'accuracy': 0.9109477124183006}]


<br>

最後に、適当な文章を読み込ませて予測を行ってみる。

In [71]:
# 7-14

text_lst = [
    "今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。",
    "昨年から黒字が減少した。",
    "今日の飲み会は楽しかった。"
]
best_model_path = checkpoint.best_model_path
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(
    best_model_path
)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


モデルで予測を行うには`BertForSequenceClassificationMultiLabel_pl.model`インスタンス変数(`BertForSequenceClassificationMultiLabel`クラス)を呼び出す必要がある(`forward`メソッドが定義されていないため？)<br>

In [67]:
type(model)

__main__.BertForSequenceClassificationMultiLabel_pl

In [68]:
type(model.model)

__main__.BertForSequenceClassificationMultiLabel

モデルをGPUに載せるときは、`model.model.cuda()`とする。

In [72]:
model = model.model.cuda()

encoding = tokenizer(
    text_lst,
    padding='longest',
    return_tensors='pt',
)
encoding = {key: value.cuda() for key, value in encoding.items()}

with torch.no_grad():
    output = model(**encoding)
scores = output.logits
labels_pred = (scores > 0).int().cpu().numpy().tolist()

In [73]:
print(f"predict: {labels_pred}")

[[1, 0, 1], [1, 0, 0], [0, 0, 0]]